In [9]:
import math

import cvxpy as cp
import scipy as sp
import numpy as np
import pickle

In [10]:
cp.installed_solvers()

['CLARABEL',
 'CVXOPT',
 'ECOS',
 'ECOS_BB',
 'GLPK',
 'GLPK_MI',
 'OSQP',
 'SCIPY',
 'SCS']

In [11]:
N = 100
M = 100
max_assigned = math.ceil(N / M)
np.random.seed(0)
W = sp.stats.lognorm.rvs(s=1, size=(N, M))
W

array([[ 5.83603919,  1.49205924,  2.66109578, ...,  5.96476998,
         1.13531721,  1.49479543],
       [ 6.57418553,  0.25982185,  0.28069545, ...,  2.27846997,
         8.69924247,  3.80580659],
       [ 0.69129969,  0.78711637,  3.00314357, ...,  1.79132161,
         0.67068947,  1.44781553],
       ...,
       [ 0.94625443,  1.13956091,  1.55378061, ...,  2.64377323,
         0.39893752,  1.94107219],
       [ 0.87503499,  0.20880169,  0.17400845, ...,  1.07341762,
        11.42413823,  2.6423831 ],
       [ 0.39338669, 17.55262551,  0.16661859, ...,  1.67677479,
         0.9676153 ,  3.66237349]], shape=(100, 100))

In [12]:
x = cp.Variable((N, M), 'x')
w = cp.Parameter((N, M), name='w')
constraints = [
    cp.sum(x, 0) <= max_assigned,   # enforce even distribution
    cp.sum(x, 1) == 1,
    0 <= x,
]
problem = cp.Problem(cp.Minimize(cp.vdot(w, x)), constraints)

# test and compile
def solve_cvxpy():
    w.value = W
    problem.solve(solver='OSQP')
    return x.value

solve_cvxpy().argmax(1)

C:\Users\volke\AppData\Local\pypoetry\Cache\virtualenvs\phantom-sc2-ZmbTkktB-py3.12\Lib\site-packages\cvxpy\reductions\solvers\solving_chain.py:254: UserWarning: Your problem has too many parameters for efficient DPP compilation. We suggest setting 'ignore_dpp = True'.
  warnings.warn(


KeyboardInterrupt: 

In [ ]:
A_ub = np.tile(np.identity(M), (1, N))
b_ub = np.full(M, max_assigned)

A_eq = np.repeat(np.identity(N), M, axis=1)
b_eq = np.full(N, 1.0)

c = W.flatten()

In [ ]:
def solve_highs():
    return sp.optimize.linprog(
        c=c,
        A_ub=A_ub,
        b_ub=b_ub,
        A_eq=A_eq,
        b_eq=b_eq,
        method="highs",
    ).x.reshape((N, M))
solve_highs().argmax(1)

In [ ]:
A_ub_sparse = sp.sparse.csr_matrix(A_ub)
A_eq_sparse = sp.sparse.csr_matrix(A_eq)

In [ ]:
def solve_interior_point():
    return sp.optimize.linprog(
        c=c,
        A_ub=A_ub,
        b_ub=b_ub,
        A_eq=A_eq,
        b_eq=b_eq,
        method="interior-point",
        options=dict(sparse=True),
    ).x.reshape((N, M))
solve_interior_point().argmax(1)

In [ ]:
%%timeit
solve_cvxpy()

In [ ]:
%%timeit
solve_highs()

In [ ]:
%%timeit
solve_interior_point()